## This script contains the minimum to build a model for the current ultrastar_pitch build

### load fft data from disk

In [ ]:
import os
import random
import numpy as np

DATA_DIRS = [#"/path/to/fft_stable",
             #"/path/to/fft_beta",
            ]

# load paths with training data
pitch_paths = []
for directory in DATA_DIRS:
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith(".npy"):
                pitch_paths.append(os.path.join(root, file))

# shuffle paths to reduce correlation
random.seed(42)
random.shuffle(pitch_paths)

# load data features and labels
X = np.array([np.load(pitch_path).flatten() for pitch_path in pitch_paths])
y = np.array([int(pitch_path.split(os.path.sep)[-2]) for pitch_path in pitch_paths])

### filter out misslabeld data

In [ ]:
# no effectiv concept so far

### apply PCA to reduce the feature dimension space

In [ ]:
from sklearn.decomposition import PCA

decomp = PCA(n_components=256)
X = decomp.fit_transform(X)

In [ ]:
# show variance after reduction
np.sum(decomp.explained_variance_ratio_)

In [ ]:
# if you want to use pca with ultrastar_pitch, it is necessary to save the parameters 
# and load them with the PCA class. pickle isn't use to save the pca-model, because I try to avoid unnecessary
# dependencies which would bloat the windows executable.
np.save("/path/to/pca_mean.npy", decomp.mean_)
np.save("/path/to/pca_components.npy", decomp.components_)

### convert the labels from integers to vectors (for 2-class, binary)

In [ ]:
# the tf model output layer is structured as an 1d-boolean-array
from sklearn.preprocessing import LabelBinarizer

lb = LabelBinarizer()
y = lb.fit_transform(y)

### split data into a training and a testing set

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state = 0)

### load tensorflow and print version

In [ ]:
import tensorflow as tf

print(tf.__version__) # version should be at least 2.0

### define the model architecture

In [ ]:
# note: the model has a 256-96-12 structure, the dropout prevents overfitting

# use a simple sequential model
model = tf.keras.Sequential()
# define input layer with 256 nodes and a weight dropout rate of 0.1
model.add(tf.keras.layers.Dropout(rate=0.1, input_shape=(256,)))
# define a hidden layer with 96 nodes and relu activation function
model.add(tf.keras.layers.Dense(96, activation="relu"))
# set the weight dropout between the last two layers to 0.3
model.add(tf.keras.layers.Dropout(rate=0.3))
# define the output layer with 12 nodes (12 pitches)
model.add(tf.keras.layers.Dense(12, activation="softmax"))

### compile the model

In [ ]:
# use adam optimizer with an initial learning rate of 0.001
opt = tf.keras.optimizers.Adam(lr=0.001)
# use categorical_crossentropy as loss function while training
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

### train the model

In [ ]:
# train the model with 30 iterations and a batch size of 32
EPOCHS, BATCH_SIZE = 30, 32
H = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=EPOCHS, batch_size=BATCH_SIZE)

### evaluate the model precision

In [ ]:
from sklearn.metrics import classification_report

# print individual and average accuracy
predictions = model.predict(X_test, batch_size=BATCH_SIZE)
print(classification_report(y_test.argmax(axis=1), predictions.argmax(axis=1)))

In [ ]:
import matplotlib.pyplot as plt

# plot the training/testing loss and accuracy
N = np.arange(0, EPOCHS)
plt.style.use("ggplot")
plt.figure()
plt.plot(N, H.history["loss"], label="train_loss")
plt.plot(N, H.history["val_loss"], label="val_loss")
plt.plot(N, H.history["accuracy"], label="train_acc")
plt.plot(N, H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy (Simple NN)")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend()

### save the trained model to disk

In [ ]:
tf.saved_model.save(model, "/path/to/tf2_256_96_12_astft_pca_X.model")